#### Importing appropriate libraries

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

#### Importing data from wikipedia

In [3]:
# Use the Notebook to build the code to scrape the following Wikipedia page, 
# https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

wikilink = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(wikilink.text,'lxml')

In [14]:
data = []
columns = []
table = soup.find(class_='wikitable')

for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
        
    if (index==0):
        columns = section
    else:
        data.append(section)

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

can_df = pd.DataFrame(data=data, columns=columns)
can_df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


#### Removing values where Borough is Not assigned

In [15]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

can_df = can_df[can_df['Borough'] != 'Not assigned']
can_df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Grouping data basis Postal Code

In [16]:
# More than one neighborhood can exist in one postal code area.

new_df = can_df.groupby("Postal Code").agg(lambda x:','.join(set(x)))
new_df.head(10)

,Borough,Neighborhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


In [17]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

new_df.loc[new_df['Neighborhood']=="Not assigned",'Neighborhood']=new_df.loc[new_df['Neighborhood']=="Not assigned",'Borough']

In [18]:
new_df.shape

(103, 2)